In [237]:
# Load libararies
import sys, os, pickle, time
import pandas
import pyomo.opt as pyomo_opt
import pyomo.environ
import pyomo.core as pyomo

In [4]:
# Define inputs
work_dir = os.getcwd()
data_dir = os.path.join(work_dir, 'data')
parcel_csv = os.path.join(data_dir, 'Parcels.txt')
adjacent_csv = os.path.join(data_dir, 'Adjacency.txt')

# Define outputs
pickle_out = os.path.join(work_dir, 'lab4_conrad.pkl')
lp_out = os.path.join(work_dir, 'lab4_conrad.lp')

In [5]:
# Load data: parcels and adjacency
parcel_df = pandas.read_csv(parcel_csv, header=None, names=['id', 'area', 'price'])
adjacent_df = pandas.read_csv(adjacent_csv, header=None, names=['id1', 'id2'])
temp = adjacent_df.groupby(['id1'])['id2'].apply(lambda x: list(x)).reset_index().rename(columns={'id1': 'id', 'id2': 'adj_list'})
parcel_df = parcel_df.merge(temp, on='id', how='left')
print(parcel_df.info())
print(parcel_df.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 0 to 119
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        120 non-null    int64  
 1   area      120 non-null    float64
 2   price     120 non-null    float64
 3   adj_list  89 non-null     object 
dtypes: float64(2), int64(1), object(1)
memory usage: 4.7+ KB
None
   id    area     price      adj_list
0   1  37.519  337671.0           NaN
1   2   6.620   19860.0          [51]
2   3  24.126   48252.0  [50, 55, 60]
3   4   5.379   80685.0    [5, 6, 48]
4   5  12.252   49008.0       [4, 54]


In [270]:
# Define functions

# Function to create params and fill with None for any missing values
def create_params(df_in, idx=None):
    df_in = df_in.dropna() # Do this so no wierd null values get added as parameters and force to be None type
    if idx is not None:
        d_out = {i: None for i in idx.value_list}
    else:
        d_out = dict()
    d_out.update(dict(df_in.values))
    return d_out

# Constraint functions
def rule_budget(model):
    return sum([model.price_i[i] * model.Cells[i,p] for i in model.sCells for p in model.sPatches]) <= model.budget_max

def rule_selected_cells(model, i, p):
    '''Mark required nodes as = 1'''
    return model.Cells[i,p] == 1

def rule_flow_on(model, i, j, p):
    '''TBD - Figure this one out again'''
    return model.EdgesAll[i,j,p] <= model.Cells[j,p] * model.cells_n

# mabye I can tighten flow-on rule by the amount coming into eacah patch source??

def rule_flow_through(model, j, p):
    '''Flow preservation constraint, leaving one behind for every Xj turned on'''
    aj = model.cells_adj_i[j]
    if aj is not None:
        return sum([model.EdgesAll[i,j,p] for i in aj]) == model.Cells[j,p] + sum([model.EdgesAll[j,l,p] for l in aj if l not in model.sSource])
    else:
        return pyomo.Constraint.Feasible

def rule_flow_balance(model, p):
    '''Sum of all nodes turned on must not exceed incoming flow to terminals'''
    return sum([model.Cells[i, p] for i in model.sCells]) == sum([model.EdgesAll[i,j,p] for i,j in model.sEdgesSource])

def rule_flow_max(model):
    '''Constrains flow through all the different patch sources to be no more than N'''
    return sum([model.Source[i] for i in model.sSource]) + sum([model.EdgesAll[i,j,p] for i,j in model.sEdgesSource for p in model.sPatches]) <= model.cells_n

def rule_source_indicator_off(model, i, j, p):
    '''Forces indicator to be off if source flow is off'''
    return model.SourceIndicator[i, j, p] <= model.EdgesAll[i, j, p]
    
def rule_source_indicator_on(model, i, j, p):
    '''Forces indicator to be on if source flow is on'''
    return model.SourceIndicator[i, j, p] >= model.EdgesAll[i, j, p] / model.cells_n

def rule_source_indicator_n(model, p):
    '''Forces each patch to only have one source'''
    return sum([model.SourceIndicator[i,j, p] for i,j in model.sEdgesSource]) <= 1

def rule_patch_one_per_cell(model, i):
    '''Forces cell to only be assigned to one single patch'''
    return sum([model.Cells[i,p] for p in model.sPatches]) <= 1

def rule_patch_no_adajcent(model, i, p):
    '''Force cells to not be adjacent, can extend to a "buffer" distance by changing adjacency set to include cells further away'''
    adj_i = model.cells_adj_i[i]
    if adj_i is not None:
        a = [model.Cells[j,p1] for p1 in model.sPatches for j in adj_i if p1 is not p and j not in model.sSource]
        a_len = len(adj_i)
    else:
        a = list()
        a_len = 1  
    return sum(a) + model.Cells[i,p] * a_len <= a_len

def rule_patch_max_area(model, p):
    '''Force patch to be less than patch max'''
    patch_a = sum([model.Cells[i,p]*model.area_i[i] for i in model.sCells])
    return patch_a <= model.patch_max_area.value

def rule_patch_min_area(model, p):
    '''Force patch to be more than patch min'''
    patch_a = sum([model.Cells[i,p]*model.area_i[i] for i in model.sCells])
    return patch_a >= model.patch_min_area.value
       
def rule_patch_off_indicator(model, p):
    return model.PatchesIndicator[p] <= sum([model.Cells[i,p] for i in model.sCells])

def rule_patch_on_indicator(model, p):
    return model.PatchesIndicator[p] >= sum([model.Cells[i,p] for i in model.sCells]) / model.cells_n

def rule_patch_n(model):
    return model.PatchesN == sum([model.PatchesIndicator[p] for p in model.sPatches])

def rule_patch_n_exact(model):
    return model.PatchesN == model.patch_n.value

def rule_patch_n_max(model):
    return model.PatchesN <= model.patch_max_n
    
def rule_patch_n_min(model):
    return model.PatchesN >= model.patch_min_n 

def rule_selected_area_min(model):
    '''Set maximum area selected less than selected upper value'''
    selected_a = sum([model.Cells[i,p]*model.area_i[i] for i in model.sCells for p in model.sPatches])
    total_a = sum([model.area_i[i] for i in model.sCells])
    return selected_a >= model.select_lower * total_a

def rule_selected_area_max(model):
    '''Set maximum reatined area equal to proportion. TBD - hard to enforce strict target, maybe a range?'''
    selected_a = sum([model.Cells[i,p]*model.area_i[i] for i in model.sCells for p in model.sPatches])
    total_a = sum([model.area_i[i] for i in model.sCells])
    return selected_a <= model.select_upper * total_a    
          
# Notes
# Adjacency set for i does not incude i

In [272]:
def create_patch_selected_cells_dict(select_cells, adj_dict):
    p = 0
    select_dict = dict()
    for c in select_cells:
        if c not in select_dict.keys():
            select_dict[c] = p
            for adj in adj_dict.get(c, list()):
                if adj in select_cells:
                    select_dict[adj] = p    
            p += 1
    return select_dict, p+1
    
def conrad_unrooted(parcel_df, adjacent_df, budget_max, patch_n=None, patch_max_n=None, patch_min_n=1, patch_max_area=None, patch_min_area=None, select_proportion=1, select_buffer=0.01, select_cells=list()):
    
    # Run some variable checks
    # TBD
    
    # Setup selection target
    assert select_proportion <=1 and select_proportion >= 0
    select_lower = select_proportion * (1 - select_buffer)
    select_upper = select_proportion * (1 + select_buffer)
    
    # Setup other variables
    source_idx = -1
    cells_n = len(parcel_df)
    source_cells = parcel_df.id.values # All cells can be sources        
    edge_list = [tuple(x) for x in adjacent_df.values]
    
    # Process selected cells, identify those touching and make select-2-patch dict
    adj_dict = create_params(parcel_df[['id', 'adj_list']])
    select_dict, select_n = create_patch_selected_cells_dict(select_cells, adj_dict)
        
    ### Maybe require a cell-patch dict from user, then could force selected cells onto same patch and do corridor problems.
    
    # Create patch list
    if patch_max_n is None:
        patch_max_n = cells_n # Defaults to at maximum a separate patch for every cell
    if patch_n is not None:
        patch_max_n = patch_n # If given specific patch N then don't have "max" any more than given
    patch_max_n = max([select_n, patch_max_n]) # Make sure that patch_max can take all given "selected" cells   
    
    #### Could add functionality to force patch_max_n to be length of selected cell list, but TBD
    
    # Define model
    model = pyomo.ConcreteModel() 
    
    # Define patches
    model.sPatches = pyomo.Set(initialize=[i for i in range(patch_max_n)])
    model.PatchesIndicator = pyomo.Var(model.sPatches, within=pyomo.Boolean, initialize=0)
 
    # Define ancillary variables and sets
    model.PatchesN = pyomo.Var(within=pyomo.NonNegativeReals, initialize=0)
    model.sSelected = pyomo.Set(initialize=select_dict.items())

    # Define node variables for every cell and patch combination
    model.sCells = pyomo.Set(initialize=parcel_df.id.values)
    model.Cells = pyomo.Var(model.sCells * model.sPatches, within=pyomo.Boolean, initialize=0)
    
    # Define primary source node
    model.sSource = pyomo.Set(initialize=[source_idx])
    model.Source = pyomo.Var(model.sSource, within=pyomo.NonNegativeReals, initialize=0)
    
    # Define indicator variables for sources
    model.sEdgesSource = pyomo.Set(initialize =(model.sSource * model.sCells))
    model.SourceIndicator = pyomo.Var(model.sEdgesSource * model.sPatches, within=pyomo.Boolean, initialize=1)

    # Define edge variables - from adajacency AND from sources
    model.sEdgesCells = pyomo.Set(initialize=edge_list, dimen=2)
    model.sEdgesAll = model.sEdgesCells | model.sEdgesSource
    model.EdgesAll = pyomo.Var(model.sEdgesAll * model.sPatches, within=pyomo.NonNegativeReals, initialize=0)
 
    # Define other parameters
    model.area_i = pyomo.Param(model.sCells, initialize=create_params(parcel_df[['id', 'area']]))
    model.price_i = pyomo.Param(model.sCells, initialize=create_params(parcel_df[['id', 'price']]))
    model.budget_max = pyomo.Param(initialize=budget_max)
    model.cells_n = pyomo.Param(initialize=cells_n)
    model.patch_max_n = pyomo.Param(initialize=patch_max_n)
    model.patch_min_n = pyomo.Param(initialize=patch_min_n)
    model.patch_n = pyomo.Param(initialize=patch_n)
    model.patch_max_area = pyomo.Param(initialize=patch_max_area)
    model.patch_min_area = pyomo.Param(initialize=patch_min_area)
    model.select_lower = pyomo.Param(initialize=select_lower)
    model.select_upper = pyomo.Param(initialize=select_upper)
                                       
    # Define adjacency sets
    adj_dict = create_params(parcel_df[['id', 'adj_list']], idx=model.sCells)
    
    # Update adjacency sets to include new source edges
    for s in source_cells:
        s_adj = adj_dict.get(s)
        if s_adj is not None:
            s_adj = s_adj + [source_idx]
        else:
            s_adj = [source_idx]
        adj_dict.update({s: s_adj})
    model.cells_adj_i = pyomo.Param(model.sCells, initialize=adj_dict)

    # Define objective
    model.obj = pyomo.Objective(expr=sum([model.area_i[i] * model.Cells[i,p] for i in model.sCells for p in model.sPatches]), sense=pyomo.maximize)

    # Define constraints
    model.c_budget = pyomo.Constraint(rule=rule_budget)
    model.c_selected = pyomo.Constraint(model.sSelected, rule=rule_selected_cells)
    model.c_flow_on = pyomo.Constraint(model.sEdgesAll*model.sPatches, rule=rule_flow_on)
    model.c_flow_through = pyomo.Constraint(model.sCells*model.sPatches, rule=rule_flow_through)
    model.c_flow_balance = pyomo.Constraint(model.sPatches, rule=rule_flow_balance)
    model.c_flow_max = pyomo.Constraint(rule=rule_flow_max)    
    model.c_source_indicator_on = pyomo.Constraint(model.sEdgesSource*model.sPatches, rule=rule_source_indicator_on)
    model.c_source_indicator_off = pyomo.Constraint(model.sEdgesSource*model.sPatches, rule=rule_source_indicator_off)
    model.c_source_indicator_n = pyomo.Constraint(model.sPatches, rule=rule_source_indicator_n)
    model.c_patch_one_per_cell = pyomo.Constraint(model.sCells, rule=rule_patch_one_per_cell)
    model.c_patch_no_adjacent = pyomo.Constraint(model.sCells*model.sPatches, rule=rule_patch_no_adajcent)
    model.c_patch_off_indicator = pyomo.Constraint(model.sPatches, rule=rule_patch_off_indicator)
    model.c_patch_on_indicator = pyomo.Constraint(model.sPatches, rule=rule_patch_on_indicator)
    model.c_patch_n = pyomo.Constraint(rule=rule_patch_n)
    model.c_patch_n_min = pyomo.Constraint(rule=rule_patch_n_min)
    model.c_patch_n_max = pyomo.Constraint(rule=rule_patch_n_max)
    if patch_n is not None:
        model.c_patch_n_exact = pyomo.Constraint(rule=rule_patch_n_exact)
    #model.c_patch_max_area = pyomo.Constraint(model.sPatches, rule=rule_patch_max_area)
    #model.c_patch_min_area = pyomo.Constraint(model.sPatches, rule=rule_patch_min_area)    
    #model.c_selected_area_max = pyomo.Constraint(rule=rule_selected_area_max)
    #model.c_selected_area_min = pyomo.Constraint(rule=rule_selected_area_min)
    
    # Return model
    return model

In [280]:
def get_active_vars(model_in, var_in, idx_in):
    idx_out = []
    for idx in idx_in:
        v = var_in[idx].value
        if v > 0:
            idx_out.append(idx)
    return idx_out

# Write model
stime = time.time()
model = conrad_unrooted(parcel_df, adjacent_df, budget_max=1000000, patch_min_n=5, patch_max_n=10)#, select_cells=[23])
print(time.time() - stime)

# Solve model with 'Gurobi'
solver = pyomo_opt.SolverFactory('gurobi')
solver.options['threads'] = 10
solution = solver.solve(model, tee=True)
print(solution)

# Results
model.obj.display()
a = get_active_vars(model, model.Cells, model.sCells*model.sPatches)
print(a)
print(len(a))

0.5174062252044678
Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-24
Read LP format model from file /tmp/tmp6pkhgeop.pyomo.lp
Reading time = 0.03 seconds
x6213: 8766 rows, 6213 columns, 54225 nonzeros
Set parameter Threads to value 10
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 10 threads

         Reduce the value of the Threads parameter to improve performance

Optimize a model with 8766 rows, 6213 columns and 54225 nonzeros
Model fingerprint: 0x13e601eb
Variable types: 3803 continuous, 2410 integer (2410 binary)
Coefficient statistics:
  Matrix range     [8e-03, 9e+05]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 172.1520000
Presolve removed 1576 rows and 653 columns
Presolve time: 0.36s
Presolved: 7190 rows, 5560 columns, 51106 nonzeros
Variable types: 3470 continuous, 209

KeyboardInterrupt: 

In [163]:
### To Do ###
# Double check constraints: 
## Does N flow injected need to change / maybe it can be restricted to N-#patches that are forced?
#### Maybe can use a "master" dummy node which can only have N flowing out of it which feeds the other "source" nodes
## Can I trim any other constraints down, probably something to do with patches increasing?

# Turn into abstract model
# Create synthetic grid-like dataset for testing of scaling power
# Work on outputs more

In [180]:
# Display results
model.obj.display()

# Write pickled object to file
with open(pickle_out, mode='wb') as file:
    pickle.dump(model, file)
model.write(lp_out, io_options={'symbolic_solver_labels':True})

obj : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 37.519
[1]
1
119.0
120
